# *Unsupervised learning: Latent Dirichlet allocation (LDA) topic modeling*

In [1]:
## Install Python package for LDA
# http://pythonhosted.org/lda/getting_started.html

!pip install lda
!pip install nltk
!pip install scikit-learn

In [18]:
## Importing basic packages

import os
import gensim
import numpy as np


INFO:summarizer.preprocessing.cleaner:'pattern' package not found; tag filters are not available for English


In [3]:
os.chdir('/Volumes/benson/Mellon/cirma_related/Topic_Modeling_Workshop_Materials/cirma_texts/')

In [4]:
document_list = []

for filename in [item for item in os.listdir('./') if '.txt' in item]:
    text_data = open(filename).read()
    document_list.append(text_data)

In [5]:
## Importing NLTK stop words
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string

 
stop_words = stopwords.words('spanish') + ["'s", "'re", '”', '“', '’', '—'] + list(string.punctuation)

string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [6]:
## Tokenizing and removing stop words from our list of documents

documents_filtered = []

for document in document_list:
    token_list = word_tokenize(document.lower())
    tokens_filtered = [item for item in token_list if (item not in stop_words)]
    documents_filtered.append(' '.join(tokens_filtered))

In [7]:
## Viewing a preprocessed document

documents_filtered[30]

"distintos lugares pais localizan ocho cadaveres baleados vejados cuatro víctimas podido ser identificadas gabinete identificación policía ocho cadáveres perforaciones bala vejámenes continuando carnicería humana sufre país algún tiempo localizados distintas partes siendo identificadas únicamente cuatro víctimas camino terracería aldea san miguel pajapa municipio pajapita conduce cantón ixcahuin municipio nuevo progreso san marcos localizado cadáver perforado tiros víctima supuestamente secuestrada identificada angel paulo orozco lópez 33 años edad siendo trasladada anfiteatro coatepeque parte carretera ciudad escuintla conduce colonia portales encontrado asesinado dos balazos tórax rostro señor andrés lópez 42 años edad autores crimen utilizaron armas calibre 38 milímetros informó mañana policía nacional realiza investigaciones terrenos finca agua caliente —propiedad señor porfirio orellana— ubicada jurisdicción monjas departamento jalapa descubierto cadáver varias perforaciones bala 

In [8]:
## Vectorizing preprocessed essays

from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()

X = vectorizer.fit_transform(documents_filtered) 

In [9]:
## Creating a vocabulary list corresponding to the vectors we created above

vocabulary = vectorizer.get_feature_names()

vocabulary[1140:1160]

['citadino',
 'citado',
 'citados',
 'citar',
 'ciudad',
 'ciudadano',
 'ciudadanos',
 'ciudadanía',
 'civil',
 'civiles',
 'clama',
 'claman',
 'clandestina',
 'clandestinas',
 'clandestino',
 'clarificado',
 'clarifique',
 'claro',
 'clase',
 'clases']

In [10]:
## Initializing an LDA model: 10 topics and 1500 iterations

import lda

model = lda.LDA(n_topics=10, n_iter=1500, random_state=1)

In [11]:
## Fitting the model using our list of vectors

model.fit(X)

INFO:lda:n_documents: 100
INFO:lda:vocab_size: 5711
INFO:lda:n_words: 22174
INFO:lda:n_topics: 10
INFO:lda:n_iter: 1500
/Users/anaconda3/envs/jupyter_usage/lib/python3.6/site-packages/lda/utils.py:55: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if sparse and not np.issubdtype(doc_word.dtype, int):
INFO:lda:<0> log likelihood: -264743
INFO:lda:<10> log likelihood: -204742
INFO:lda:<20> log likelihood: -201040
INFO:lda:<30> log likelihood: -198871
INFO:lda:<40> log likelihood: -197530
INFO:lda:<50> log likelihood: -196882
INFO:lda:<60> log likelihood: -196205
INFO:lda:<70> log likelihood: -195863
INFO:lda:<80> log likelihood: -195257
INFO:lda:<90> log likelihood: -195037
INFO:lda:<100> log likelihood: -194634
INFO:lda:<110> log likelihood: -194547
INFO:lda:<120> log likelihood: -194444
INFO:lda:<130> log likelihood: -194068
INFO:lda:<140> log likelihood:

In [12]:
## Viewing the top 50 words in each 'topic'

topic_word = model.topic_word_

n_top_words = 10

for i, topic_distribution in enumerate(topic_word):
    topic_words = np.array(vocabulary)[np.argsort(topic_distribution)][:-(n_top_words+1):-1]
    print('Topic ' + str(i) + ':')
    print(' '.join(topic_words))
    print()

Topic 0:
pick up camión vehículo ruta zona secuestradores estudiante edad placas

Topic 1:
años edad sido cadáver autoridades escuintla balazos cadáveres víctimas san

Topic 2:
policía colonia nacional agentes estudiante carlos tres corado cuerpo policías

Topic 3:
policía dos señor desconocidos según varios ayer lugar muerte autoridades

Topic 4:
hombres aldea campesinos armados verde grupo vecinos olivo departamento miembros

Topic 5:
alcalde iztapa bar cruz ortega puerto luis gonzález hospital lópez

Topic 6:
atentado alcalde detectives herido cuerpo casa zona guerra hospital nacional

Topic 7:
vecinos presidente terreno escuela empresa problema sector instalaciones diciendo comité

Topic 8:
ser puede problemas señor cantidad casos viene hacer oficina agua

Topic 9:
pues zona hombre ejército llegó familia hace evitar gobierno militares



In [13]:
#Make stopwords in a single variable and make a class that 
#reprocesses the documents into a dictionary for gensim format with tokenization. 
import logging 
stops = '/Users/jgo384/nltk_data/corpora/stopwords/spanish.txt'

stoplist = open(stops).read()
stoplist = stoplist.split('\n')
class MyCorpus(object):

    def __init__(self, topdir, stoplist):
        self.topdir = topdir
        self.stoplist = stoplist
        self.dictionary = gensim.corpora.Dictionary(iter_docs(topdir, stoplist))
        
    def __iter__(self):
        for tokens in iter_docs(self.topdir, self.stoplist):
            yield self.dictionary.doc2bow(tokens)


logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', 
                    level=logging.INFO)

In [20]:
#separate function to iterate through the documents separately to tokenize and preprocess with lower and accent remove
def iter_docs(topdir, stoplist):
    for fn in os.listdir(topdir):
        fin = open(os.path.join(topdir, fn), 'rb')
        text = fin.read()
        fin.close()
        yield (x for x in 
            gensim.utils.tokenize(text, lowercase=True, deacc=True, 
                                  errors="ignore")
            if x not in stoplist)

In [21]:
os.chdir('/Volumes/benson/Mellon/cirma_related/Topic_Modeling_Workshop_Materials/jupyter_notebooks/')

In [22]:
#gensim doesn't like the texts directory to be two separate so they were combined
#try: 
#os.mkdir('/Volumes/benson/Mellon/cirma_related/Topic_Modeling_Workshop_Materials/jupyter_notebooks/models/dir/')
#except: pass
TEXTS_DIR = "/Volumes/benson/Mellon/cirma_related/Topic_Modeling_Workshop_Materials/cirma_texts/"
MODELS_DIR = "/Volumes/benson/Mellon/cirma_related/Topic_Modeling_Workshop_Materials/jupyter_notebooks/models/dir"

In [23]:
#the fist is a bit of overkill since it reapplies stopwords, it also generates tokenization again
#the issue is that I can't bring the other variables over because it dislikes nltk's tokenization for utf-8
#the second creates a dictionary which is just a id per word list
#the third creates pairings between document, id, and word with a frequency count in a dense matrix
corpus = MyCorpus(TEXTS_DIR, stoplist)
corpus.dictionary.save(os.path.join(MODELS_DIR, 'cirma.dict'))
gensim.corpora.MmCorpus.serialize(os.path.join(MODELS_DIR, "cirma.mm"), 
                                  corpus)

INFO:gensim.corpora.dictionary:adding document #0 to Dictionary(0 unique tokens: [])
INFO:gensim.corpora.dictionary:built Dictionary(5411 unique tokens: ['bud', 'dsdb', 'acerca', 'acercarse', 'afrontando']...) from 101 documents (total 21943 corpus positions)
INFO:gensim.utils:saving Dictionary object under /Volumes/benson/Mellon/cirma_related/Topic_Modeling_Workshop_Materials/jupyter_notebooks/models/dir/cirma.dict, separately None
INFO:gensim.utils:saved /Volumes/benson/Mellon/cirma_related/Topic_Modeling_Workshop_Materials/jupyter_notebooks/models/dir/cirma.dict
INFO:gensim.corpora.mmcorpus:storing corpus in Matrix Market format to /Volumes/benson/Mellon/cirma_related/Topic_Modeling_Workshop_Materials/jupyter_notebooks/models/dir/cirma.mm
INFO:gensim.matutils:saving sparse matrix to /Volumes/benson/Mellon/cirma_related/Topic_Modeling_Workshop_Materials/jupyter_notebooks/models/dir/cirma.mm
INFO:gensim.matutils:PROGRESS: saving document #0
INFO:gensim.matutils:saved 101x5411 matrix, 

In [24]:
#logging just to keep track of time for these, much slower than Mallet or lda
#dictionary gets loaded from dict file
#corpus is loaded from path and linked to matrix of values for frequency, id and doc
#50 topics seems to work best and keeping the passes(iterations) at the same number as Mallet (painfully slow)
numtopics = 50
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', 
                    level=logging.INFO)

dictionary = gensim.corpora.Dictionary.load(os.path.join(MODELS_DIR, 
                                            "cirma.dict"))
corpus = gensim.corpora.MmCorpus(os.path.join(MODELS_DIR, "cirma.mm"))

# Project to LDA space
polylingualmodel = gensim.models.LdaMulticore(corpus, id2word=dictionary, num_topics=numtopics, 
                                              workers=8, random_state=1, passes=1000)

INFO:gensim.utils:loading Dictionary object from /Volumes/benson/Mellon/cirma_related/Topic_Modeling_Workshop_Materials/jupyter_notebooks/models/dir/cirma.dict
INFO:gensim.utils:loaded /Volumes/benson/Mellon/cirma_related/Topic_Modeling_Workshop_Materials/jupyter_notebooks/models/dir/cirma.dict
INFO:gensim.corpora.indexedcorpus:loaded corpus index from /Volumes/benson/Mellon/cirma_related/Topic_Modeling_Workshop_Materials/jupyter_notebooks/models/dir/cirma.mm.index
INFO:gensim.corpora._mmreader:initializing cython corpus reader from /Volumes/benson/Mellon/cirma_related/Topic_Modeling_Workshop_Materials/jupyter_notebooks/models/dir/cirma.mm
INFO:gensim.corpora._mmreader:accepted corpus with 101 documents, 5411 features, 15602 non-zero entries
INFO:gensim.models.ldamodel:using symmetric alpha at 0.02
INFO:gensim.models.ldamodel:using symmetric eta at 0.02
INFO:gensim.models.ldamodel:using serial LDA version on this node
INFO:gensim.models.ldamulticore:running online LDA training, 50 topi

INFO:gensim.models.ldamodel:topic diff=1.388307, rho=0.444972
INFO:gensim.models.ldamodel:-8.603 per-word bound, 388.9 perplexity estimate based on a held-out corpus of 101 documents with 21943 words
INFO:gensim.models.ldamulticore:PROGRESS: pass 5, dispatched chunk #0 = documents up to #101/101, outstanding queue size 1
INFO:gensim.models.ldamodel:topic #7 (0.020): 0.001*"policia" + 0.001*"alcalde" + 0.001*"varios" + 0.000*"estudiantes" + 0.000*"quevedo" + 0.000*"anos" + 0.000*"mixco" + 0.000*"personas" + 0.000*"calle" + 0.000*"jorge"
INFO:gensim.models.ldamodel:topic #17 (0.020): 0.013*"rodriguez" + 0.010*"tres" + 0.009*"senora" + 0.009*"policia" + 0.008*"joven" + 0.008*"bar" + 0.008*"segun" + 0.008*"autoridades" + 0.008*"investigaciones" + 0.007*"aun"
INFO:gensim.models.ldamodel:topic #13 (0.020): 0.018*"anos" + 0.017*"vecinos" + 0.013*"cadaver" + 0.013*"edad" + 0.011*"cordon" + 0.010*"senora" + 0.010*"bala" + 0.009*"policia" + 0.009*"herida" + 0.008*"identificado"
INFO:gensim.model

INFO:gensim.models.ldamodel:topic #3 (0.020): 0.024*"joven" + 0.017*"secuestrada" + 0.013*"pick" + 0.012*"diaz" + 0.012*"up" + 0.012*"emergencia" + 0.010*"murio" + 0.010*"miembro" + 0.010*"calderon" + 0.009*"habia"
INFO:gensim.models.ldamodel:topic diff=0.162966, rho=0.300822
INFO:gensim.models.ldamodel:-8.318 per-word bound, 319.2 perplexity estimate based on a held-out corpus of 101 documents with 21943 words
INFO:gensim.models.ldamulticore:PROGRESS: pass 11, dispatched chunk #0 = documents up to #101/101, outstanding queue size 1
INFO:gensim.models.ldamodel:topic #46 (0.020): 0.019*"instalaciones" + 0.011*"dos" + 0.009*"policia" + 0.008*"mismos" + 0.007*"agentes" + 0.007*"estudiantes" + 0.006*"amigos" + 0.006*"ley" + 0.006*"crimen" + 0.006*"corado"
INFO:gensim.models.ldamodel:topic #8 (0.020): 0.017*"anos" + 0.013*"dos" + 0.013*"escuintla" + 0.013*"tarde" + 0.013*"cadaveres" + 0.009*"hombres" + 0.009*"ayer" + 0.009*"nacional" + 0.009*"hospital" + 0.009*"victimas"
INFO:gensim.models.

INFO:gensim.models.ldamodel:topic #49 (0.020): 0.019*"camion" + 0.015*"terreno" + 0.015*"familias" + 0.013*"cemento" + 0.013*"tractores" + 0.011*"municipalidad" + 0.011*"machetes" + 0.008*"zona" + 0.008*"armados" + 0.007*"policia"
INFO:gensim.models.ldamodel:topic #43 (0.020): 0.016*"campesinos" + 0.016*"sacaron" + 0.010*"secuestradores" + 0.008*"rumbo" + 0.008*"ignorado" + 0.008*"aseguraron" + 0.008*"estacionado" + 0.008*"poblaciones" + 0.008*"aguilar" + 0.008*"vestidos"
INFO:gensim.models.ldamodel:topic diff=0.031284, rho=0.242176
INFO:gensim.models.ldamodel:-8.287 per-word bound, 312.3 perplexity estimate based on a held-out corpus of 101 documents with 21943 words
INFO:gensim.models.ldamulticore:PROGRESS: pass 17, dispatched chunk #0 = documents up to #101/101, outstanding queue size 1
INFO:gensim.models.ldamodel:topic #42 (0.020): 0.014*"isen" + 0.012*"campesinos" + 0.011*"senor" + 0.010*"hombres" + 0.010*"grupo" + 0.009*"policia" + 0.009*"san" + 0.008*"secuestradores" + 0.008*"ro

INFO:gensim.models.ldamodel:topic #17 (0.020): 0.014*"rodriguez" + 0.010*"segun" + 0.010*"tres" + 0.010*"policia" + 0.009*"senora" + 0.009*"joven" + 0.009*"bar" + 0.009*"investigaciones" + 0.009*"sido" + 0.008*"autoridades"
INFO:gensim.models.ldamodel:topic #11 (0.020): 0.015*"policia" + 0.014*"tiroteo" + 0.013*"varios" + 0.012*"familia" + 0.012*"esquipulas" + 0.011*"responsables" + 0.009*"poblacion" + 0.009*"casa" + 0.009*"ahora" + 0.008*"testigos"
INFO:gensim.models.ldamodel:topic #32 (0.020): 0.014*"azucar" + 0.012*"desconocidos" + 0.011*"edad" + 0.010*"anos" + 0.010*"cadaveres" + 0.010*"dos" + 0.009*"bala" + 0.009*"victimas" + 0.009*"precio" + 0.008*"policia"
INFO:gensim.models.ldamodel:topic diff=0.008946, rho=0.208286
INFO:gensim.models.ldamodel:-8.276 per-word bound, 310.0 perplexity estimate based on a held-out corpus of 101 documents with 21943 words
INFO:gensim.models.ldamulticore:PROGRESS: pass 23, dispatched chunk #0 = documents up to #101/101, outstanding queue size 1
INFO

...

INFO:gensim.models.ldamodel:topic #7 (0.020): 0.000*"vigilar" + 0.000*"verdadero" + 0.000*"nutrido" + 0.000*"fatal" + 0.000*"entablandose" + 0.000*"carceles" + 0.000*"carcel" + 0.000*"bar" + 0.000*"vuelta" + 0.000*"publicas"
INFO:gensim.models.ldamodel:topic #36 (0.020): 0.000*"vigilar" + 0.000*"verdadero" + 0.000*"nutrido" + 0.000*"fatal" + 0.000*"entablandose" + 0.000*"carceles" + 0.000*"carcel" + 0.000*"bar" + 0.000*"vuelta" + 0.000*"publicas"
INFO:gensim.models.ldamodel:topic #37 (0.020): 0.029*"iztapa" + 0.020*"puerto" + 0.018*"policia" + 0.017*"alcalde" + 0.016*"heridos" + 0.015*"bar" + 0.014*"roca" + 0.014*"lugar" + 0.014*"general" + 0.013*"linda"
INFO:gensim.models.ldamodel:topic #48 (0.020): 0.020*"cordon" + 0.015*"anos" + 0.014*"grupo" + 0.013*"aldea" + 0.011*"policia" + 0.010*"ayer" + 0.009*"muerte" + 0.009*"reforma" + 0.009*"dos" + 0.008*"finca"
INFO:gensim.models.ldamodel:topic diff=0.000006, rho=0.032041
INFO:gensim.models.ldamodel:-8.250 per-word bound, 304.5 perplexity 

INFO:gensim.models.ldamodel:-8.250 per-word bound, 304.5 perplexity estimate based on a held-out corpus of 101 documents with 21943 words
INFO:gensim.models.ldamulticore:PROGRESS: pass 979, dispatched chunk #0 = documents up to #101/101, outstanding queue size 1
INFO:gensim.models.ldamodel:topic #36 (0.020): 0.000*"vigilar" + 0.000*"verdadero" + 0.000*"nutrido" + 0.000*"fatal" + 0.000*"entablandose" + 0.000*"carceles" + 0.000*"carcel" + 0.000*"bar" + 0.000*"vuelta" + 0.000*"publicas"
INFO:gensim.models.ldamodel:topic #7 (0.020): 0.000*"vigilar" + 0.000*"verdadero" + 0.000*"nutrido" + 0.000*"fatal" + 0.000*"entablandose" + 0.000*"carceles" + 0.000*"carcel" + 0.000*"bar" + 0.000*"vuelta" + 0.000*"publicas"
INFO:gensim.models.ldamodel:topic #5 (0.020): 0.000*"vigilar" + 0.000*"verdadero" + 0.000*"nutrido" + 0.000*"fatal" + 0.000*"entablandose" + 0.000*"carceles" + 0.000*"carcel" + 0.000*"bar" + 0.000*"vuelta" + 0.000*"publicas"
INFO:gensim.models.ldamodel:topic #48 (0.020): 0.020*"cordon"

INFO:gensim.models.ldamodel:topic #24 (0.020): 0.018*"audiencia" + 0.018*"presidente" + 0.013*"dos" + 0.012*"garcia" + 0.012*"ruta" + 0.009*"lucas" + 0.009*"familiares" + 0.009*"rivera" + 0.009*"atlantico" + 0.009*"comite"
INFO:gensim.models.ldamodel:topic diff=0.000006, rho=0.031862
INFO:gensim.models.ldamodel:-8.250 per-word bound, 304.5 perplexity estimate based on a held-out corpus of 101 documents with 21943 words
INFO:gensim.models.ldamulticore:PROGRESS: pass 985, dispatched chunk #0 = documents up to #101/101, outstanding queue size 1
INFO:gensim.models.ldamodel:topic #16 (0.020): 0.011*"desconocidos" + 0.010*"horas" + 0.009*"alcalde" + 0.009*"atentado" + 0.009*"puerta" + 0.008*"pick" + 0.008*"anos" + 0.008*"ser" + 0.007*"up" + 0.007*"llevaron"
INFO:gensim.models.ldamodel:topic #32 (0.020): 0.016*"edad" + 0.013*"desconocidos" + 0.013*"anos" + 0.012*"azucar" + 0.012*"bala" + 0.011*"victimas" + 0.011*"balazos" + 0.010*"municipio" + 0.010*"perforaciones" + 0.010*"policia"
INFO:gens

INFO:gensim.models.ldamodel:topic #20 (0.020): 0.022*"terreno" + 0.019*"vecinos" + 0.015*"asentamiento" + 0.015*"nueva" + 0.015*"presidente" + 0.015*"banvi" + 0.015*"tierra" + 0.011*"sinegub" + 0.011*"valor" + 0.011*"carolingia"
INFO:gensim.models.ldamodel:topic #27 (0.020): 0.014*"juan" + 0.014*"taxista" + 0.014*"espalda" + 0.014*"servicio" + 0.014*"lopez" + 0.010*"hacia" + 0.010*"dos" + 0.010*"norte" + 0.010*"lomas" + 0.010*"estacionamiento"
INFO:gensim.models.ldamodel:topic #15 (0.020): 0.020*"policia" + 0.011*"romero" + 0.011*"oficina" + 0.009*"desconocidos" + 0.009*"nacional" + 0.009*"parte" + 0.007*"locutor" + 0.007*"ciudad" + 0.007*"objeto" + 0.007*"chiquimula"
INFO:gensim.models.ldamodel:topic diff=0.000006, rho=0.031765
INFO:gensim.models.ldamodel:-8.250 per-word bound, 304.5 perplexity estimate based on a held-out corpus of 101 documents with 21943 words
INFO:gensim.models.ldamulticore:PROGRESS: pass 991, dispatched chunk #0 = documents up to #101/101, outstanding queue size 

INFO:gensim.models.ldamodel:topic #45 (0.020): 0.025*"senor" + 0.015*"cuyotenango" + 0.015*"inde" + 0.015*"lectura" + 0.015*"contadores" + 0.010*"nacional" + 0.010*"situacion" + 0.010*"electrica" + 0.010*"ruiz" + 0.010*"mes"
INFO:gensim.models.ldamodel:topic #38 (0.020): 0.020*"caserio" + 0.015*"jalapa" + 0.012*"heridos" + 0.012*"cacao" + 0.012*"mateo" + 0.012*"armas" + 0.010*"oriente" + 0.010*"asaltantes" + 0.010*"olivo" + 0.010*"lopez"
INFO:gensim.models.ldamodel:topic #17 (0.020): 0.014*"rodriguez" + 0.010*"segun" + 0.010*"senora" + 0.009*"policia" + 0.009*"tres" + 0.009*"joven" + 0.009*"bar" + 0.009*"sido" + 0.009*"investigaciones" + 0.008*"autoridades"
INFO:gensim.models.ldamodel:topic #40 (0.020): 0.019*"policia" + 0.013*"hombres" + 0.012*"nacional" + 0.012*"despues" + 0.012*"anos" + 0.010*"dos" + 0.010*"agentes" + 0.010*"hospital" + 0.009*"segun" + 0.009*"senor"
INFO:gensim.models.ldamodel:topic #21 (0.020): 0.000*"vigilar" + 0.000*"verdadero" + 0.000*"nutrido" + 0.000*"fatal" +

In [25]:
#saving the model for persistency
polylingualmodel.save('polylingualmodel.model')

INFO:gensim.utils:saving LdaState object under polylingualmodel.model.state, separately None
INFO:gensim.utils:saved polylingualmodel.model.state
INFO:gensim.utils:saving LdaMulticore object under polylingualmodel.model, separately ['expElogbeta', 'sstats']
INFO:gensim.utils:storing np array 'expElogbeta' to polylingualmodel.model.expElogbeta.npy
INFO:gensim.utils:not storing attribute state
INFO:gensim.utils:not storing attribute dispatcher
INFO:gensim.utils:not storing attribute id2word
INFO:gensim.utils:saved polylingualmodel.model


In [28]:
#creating a bag-of-words from the corpus with doc id, word, frequency count array
from gensim.corpora.mmcorpus import MmCorpus
bow_corpus = MmCorpus("/Volumes/benson/Mellon/cirma_related/Topic_Modeling_Workshop_Materials/jupyter_notebooks/models/dir/cirma.mm")

INFO:gensim.corpora.indexedcorpus:loaded corpus index from /Volumes/benson/Mellon/cirma_related/Topic_Modeling_Workshop_Materials/jupyter_notebooks/models/dir/cirma.mm.index
INFO:gensim.corpora._mmreader:initializing cython corpus reader from /Volumes/benson/Mellon/cirma_related/Topic_Modeling_Workshop_Materials/jupyter_notebooks/models/dir/cirma.mm
INFO:gensim.corpora._mmreader:accepted corpus with 101 documents, 5411 features, 15602 non-zero entries


In [29]:
#This shows a random sorting of topics with the weight across the entire corpus 
#"topic distribution for the given document bow, as a list of (topic_id, topic_probability) 2-tuples"
get_documents_topics = polylingualmodel.get_document_topics(bow_corpus, 
                                                            minimum_probability=.05, 
                                                            minimum_phi_value=0.01, 
                                                            per_word_topics=True)
get_documents_topics[1]

([(31, 0.9934667)],
 [(2, [31]),
  (3, [31]),
  (4, [31]),
  (5, [31]),
  (6, [31]),
  (7, [31]),
  (8, [31]),
  (9, [31]),
  (10, [31]),
  (11, [31]),
  (12, [31]),
  (13, [31]),
  (14, [31]),
  (15, [31]),
  (16, [31]),
  (17, [31]),
  (18, [31]),
  (19, [31]),
  (20, [31]),
  (21, [31]),
  (22, [31]),
  (23, [31]),
  (24, [31]),
  (25, [31]),
  (26, [31]),
  (27, [31]),
  (28, [31]),
  (29, [31]),
  (30, [31]),
  (31, [31]),
  (32, [31]),
  (33, [31]),
  (34, [31]),
  (35, [31]),
  (36, [31]),
  (37, [31]),
  (38, [31]),
  (39, [31]),
  (40, [31]),
  (41, [31]),
  (42, [31]),
  (43, [31]),
  (44, [31]),
  (45, [31]),
  (46, [31]),
  (47, [31]),
  (48, [31]),
  (49, [31]),
  (50, [31]),
  (51, [31]),
  (52, [31]),
  (53, [31]),
  (54, [31]),
  (55, [31]),
  (56, [31]),
  (57, [31]),
  (58, [31]),
  (59, [31]),
  (60, [31]),
  (61, [31]),
  (62, [31]),
  (63, [31]),
  (64, [31]),
  (65, [31]),
  (66, [31]),
  (67, [31]),
  (68, [31]),
  (69, [31]),
  (70, [31]),
  (71, [31]),
  (72, [

In [30]:
#This prints a topic at a time and is useful for inspecting topics one by one. The one listed here
#bears some slight resemblance to 21 in Mallet
print_topics = polylingualmodel.print_topics(num_topics=1, num_words=35)

INFO:gensim.models.ldamodel:topic #41 (0.020): 0.014*"diaz" + 0.011*"pick" + 0.011*"up" + 0.010*"secuestradores" + 0.009*"colonia" + 0.008*"horas" + 0.008*"haidee" + 0.008*"gloria" + 0.008*"luego" + 0.007*"senora" + 0.007*"segun" + 0.006*"tres" + 0.006*"estudiante" + 0.006*"casa" + 0.006*"banvi" + 0.006*"vecinos" + 0.006*"maria" + 0.006*"accidente" + 0.006*"cecilia" + 0.006*"calderon" + 0.006*"camion" + 0.005*"vehiculo" + 0.005*"autoridades" + 0.005*"violeta" + 0.005*"salieron" + 0.005*"juan" + 0.005*"hacia" + 0.005*"anos" + 0.005*"habia" + 0.005*"san" + 0.005*"policia" + 0.004*"garcia" + 0.004*"secuestro" + 0.004*"dos" + 0.004*"senalo"


In [31]:
#From gensim site "tuples with (topic_repr, coherence_score), where topic_repr is a list of 
#representations of the topn terms for the topic. 
#The terms are represented as tuples of (membership_in_topic, token). 
#The coherence_score is a float".
top_topics = polylingualmodel.top_topics(corpus=corpus, texts=None, dictionary='cirma.dict', 
                                         window_size=None, coherence='u_mass', 
                                         topn=50, processes=-1)
top_topics

[([(0.02881508, 'iztapa'),
   (0.020073783, 'puerto'),
   (0.018226288, 'policia'),
   (0.016561288, 'alcalde'),
   (0.016403966, 'heridos'),
   (0.014662028, 'bar'),
   (0.014057665, 'roca'),
   (0.013961678, 'lugar'),
   (0.013528971, 'general'),
   (0.013054981, 'linda'),
   (0.013042369, 'hospital'),
   (0.011301895, 'ayer'),
   (0.011228034, 'tiroteo'),
   (0.010215882, 'cruz'),
   (0.010046918, 'cobrador'),
   (0.009876789, 'capital'),
   (0.009764542, 'dos'),
   (0.009020935, 'personas'),
   (0.008677042, 'muertos'),
   (0.0085946005, 'esperanza'),
   (0.008412015, 'luis'),
   (0.008323948, 'ortega'),
   (0.008142771, 'municipal'),
   (0.008046808, 'cesar'),
   (0.00804154, 'balacera'),
   (0.007930711, 'hombres'),
   (0.007870758, 'armados'),
   (0.007774982, 'augusto'),
   (0.007705203, 'mendez'),
   (0.007574074, 'bonilla'),
   (0.00754683, 'mas'),
   (0.0074564987, 'aquino'),
   (0.0070388587, 'juez'),
   (0.006958278, 'minutos'),
   (0.0067101857, 'gonzalez'),
   (0.0067091

In [32]:
#even with pprint this is quite difficult to sort through but there seems to be an issue with the weights
#all being too low at .00*
from pprint import pprint
pprint(top_topics)

[([(0.02881508, 'iztapa'),
   (0.020073783, 'puerto'),
   (0.018226288, 'policia'),
   (0.016561288, 'alcalde'),
   (0.016403966, 'heridos'),
   (0.014662028, 'bar'),
   (0.014057665, 'roca'),
   (0.013961678, 'lugar'),
   (0.013528971, 'general'),
   (0.013054981, 'linda'),
   (0.013042369, 'hospital'),
   (0.011301895, 'ayer'),
   (0.011228034, 'tiroteo'),
   (0.010215882, 'cruz'),
   (0.010046918, 'cobrador'),
   (0.009876789, 'capital'),
   (0.009764542, 'dos'),
   (0.009020935, 'personas'),
   (0.008677042, 'muertos'),
   (0.0085946005, 'esperanza'),
   (0.008412015, 'luis'),
   (0.008323948, 'ortega'),
   (0.008142771, 'municipal'),
   (0.008046808, 'cesar'),
   (0.00804154, 'balacera'),
   (0.007930711, 'hombres'),
   (0.007870758, 'armados'),
   (0.007774982, 'augusto'),
   (0.007705203, 'mendez'),
   (0.007574074, 'bonilla'),
   (0.00754683, 'mas'),
   (0.0074564987, 'aquino'),
   (0.0070388587, 'juez'),
   (0.006958278, 'minutos'),
   (0.0067101857, 'gonzalez'),
   (0.0067091

   (0.0057906266, 'ayer'),
   (0.00577744, 'policia'),
   (0.0057487064, 'miembros'),
   (0.0050510718, 'cuenta'),
   (0.004989829, 'aguas'),
   (0.004918821, 'concejo'),
   (0.0049188207, 'lugar'),
   (0.0048746057, 'agua'),
   (0.004171422, 'aldea'),
   (0.003987759, 'pues'),
   (0.003938975, 'agente'),
   (0.003938975, 'proyecto'),
   (0.003938974, 'comite'),
   (0.0039342106, 'problema'),
   (0.0039244117, 'hora'),
   (0.0039189663, 'dieron'),
   (0.0039109443, 'campesinos'),
   (0.0038986686, 'puede'),
   (0.0038578704, 'vecinos'),
   (0.003733876, 'parte'),
   (0.0037307139, 'habia'),
   (0.0034686592, 'varios'),
   (0.003440625, 'ciudad'),
   (0.0034388718, 'salud'),
   (0.0032385883, 'dos'),
   (0.0030430197, 'ninguna'),
   (0.0030044834, 'senor'),
   (0.0029943914, 'chimaltenango'),
   (0.002970049, 'empresa'),
   (0.0029591317, 'darse'),
   (0.0029591317, 'mayoria'),
   (0.0029591317, 'custodio'),
   (0.0029591317, 'grimler'),
   (0.0029591317, 'leon'),
   (0.0029591317, 'cam

In [43]:
#os.chdir('/Volumes/benson/Mellon/cirma_related/Topic_Modeling_Workshop_Materials/jupyter_notebooks/models/dir')
d = gensim.corpora.Dictionary.load('/Volumes/benson/Mellon/cirma_related/Topic_Modeling_Workshop_Materials/jupyter_notebooks/models/dir/cirma.dict')
c = gensim.corpora.MmCorpus('/Volumes/benson/Mellon/cirma_related/Topic_Modeling_Workshop_Materials/jupyter_notebooks/models/dir/cirma.mm')
vizmod = gensim.models.LdaMulticore.load('/Volumes/benson/Mellon/cirma_related/Topic_Modeling_Workshop_Materials/jupyter_notebooks/polylingualmodel.model')

INFO:gensim.utils:loading Dictionary object from /Volumes/benson/Mellon/cirma_related/Topic_Modeling_Workshop_Materials/jupyter_notebooks/models/dir/cirma.dict
INFO:gensim.utils:loaded /Volumes/benson/Mellon/cirma_related/Topic_Modeling_Workshop_Materials/jupyter_notebooks/models/dir/cirma.dict
INFO:gensim.corpora.indexedcorpus:loaded corpus index from /Volumes/benson/Mellon/cirma_related/Topic_Modeling_Workshop_Materials/jupyter_notebooks/models/dir/cirma.mm.index
INFO:gensim.corpora._mmreader:initializing cython corpus reader from /Volumes/benson/Mellon/cirma_related/Topic_Modeling_Workshop_Materials/jupyter_notebooks/models/dir/cirma.mm
INFO:gensim.corpora._mmreader:accepted corpus with 101 documents, 5411 features, 15602 non-zero entries
INFO:gensim.utils:loading LdaMulticore object from /Volumes/benson/Mellon/cirma_related/Topic_Modeling_Workshop_Materials/jupyter_notebooks/polylingualmodel.model
INFO:gensim.utils:loading expElogbeta from /Volumes/benson/Mellon/cirma_related/Topic

In [50]:
#!pip install pyldavis
import pyLDAvis
import pyLDAvis.gensim

pyLDAvis.enable_notebook()
data = pyLDAvis.gensim.prepare(vizmod, c, d)
data

/Users/anaconda3/envs/jupyter_usage/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
10    -0.156763 -0.081307       1        1  6.750947
0     -0.126728  0.071987       2        1  6.403624
35    -0.153461 -0.106990       3        1  4.712314
22    -0.057897 -0.045197       4        1  4.361896
31    -0.101692 -0.066668       5        1  4.277660
18    -0.035126  0.026641       6        1  4.149812
37    -0.166387  0.168283       7        1  4.036606
41    -0.094178 -0.150359       8        1  3.870823
40    -0.180775 -0.018459       9        1  3.773386
42    -0.085041  0.080727      10        1  3.675357
4     -0.134813 -0.028809      11        1  3.530643
6     -0.112002  0.107081      12        1  3.156086
23    -0.138112 -0.059712      13        1  3.153141
25    -0.015449  0.090188      14        1  3.014827
32    -0.097627  0.020772      15        1  2.865889
44    -0.049231  0.017104      16        1  2.824018
16    -0.120609 -0.048692      17        1  2.603122
46     0.016556 -0.066037      18        1  2.319886
13    -0.058784 -0.046031      19        1  2.046045
17    -0.010122 -0.060939      20        1  2.028585
49    -0.021977 -0.011843      21        1  1.958537
2     -0.008683  0.102055      22        1  1.957258
39    -0.055785  0.045797      23        1  1.831322
47    -0.017153  0.035362      24        1  1.794092
48    -0.032213  0.001879      25        1  1.539029
12    -0.015510 -0.111051      26        1  1.516610
15    -0.045327  0.071371      27        1  1.508587
19     0.010782  0.050807      28        1  1.457511
11    -0.008968  0.060300      29        1  1.448755
45     0.083683  0.014613      30        1  1.358084
38     0.016125  0.125463      31        1  1.349061
3      0.019229 -0.110590      32        1  1.340336
28     0.042169 -0.114176      33        1  1.254227
33     0.024868  0.099358      34        1  1.142588
24     0.079107 -0.009557      35        1  1.020362
1      0.083902 -0.037380      36        1  0.831241
20     0.134535 -0.006745      37        1  0.718619
8      0.073832  0.024730      38        1  0.556027
34     0.101358  0.021646      39        1  0.504897
9      0.163258 -0.028344      40        1  0.494970
27     0.090591 -0.010949      41        1  0.474003
14     0.130692 -0.015462      42        1  0.252986
43     0.135618 -0.000281      43        1  0.072406
36     0.127730 -0.000084      44        1  0.009119
7      0.127730 -0.000084      45        1  0.009119
21     0.127730 -0.000084      46        1  0.009119
5      0.127730 -0.000084      47        1  0.009119
30     0.127730 -0.000084      48        1  0.009119
29     0.127730 -0.000084      49        1  0.009119
26     0.127730 -0.000084      50        1  0.009119, topic_info=     Category       Freq          Term       Total  loglift  logprob
term                                                                
341   Default   94.00000       alcalde   94.000000  30.0000  30.0000
323   Default   94.00000         senor   94.000000  29.0000  29.0000
252   Default   84.00000          anos   84.000000  28.0000  28.0000
164   Default   50.00000    estudiante   50.000000  27.0000  27.0000
4517  Default   35.00000        iztapa   35.000000  26.0000  26.0000
36    Default  111.00000           dos  111.000000  25.0000  25.0000
3607  Default   31.00000           bar   31.000000  24.0000  24.0000
862   Default   27.00000        guerra   27.000000  23.0000  23.0000
526   Default   35.00000        garcia   35.000000  22.0000  22.0000
89    Default  220.00000       policia  220.000000  21.0000  21.0000
153   Default   73.00000  desconocidos   73.000000  20.0000  20.0000
2004  Default   22.00000   estudiantes   22.000000  19.0000  19.0000
273   Default   50.00000          edad   50.000000  18.0000  18.0000
1799  Default   37.00000        camion   37.000000  17.0000  17.0000
245   Default   90.00000          zona   90.000000  16.0000  16.0000
334   Default   60.00000    